# 14.9 物体検知

◆物体検知とは(object detection)<br>
 1つの画像に含まれる複数の物体を分類し、位置を特定するタスク

◆参考URL<br>
[https://aiacademy.jp/media/?p=1173](https://aiacademy.jp/media/?p=1173)

物体検出は主に以下の２ステップで行われます。<br>
1.対象物体が画像中のどこにありそうかを絞り込む（バウンディングボックスの発見）<br>
2.対象物体が何であるかを分類（カテゴリー判断）

◆参考<br>
[https://qiita.com/mshinoda88/items/9770ee671ea27f2c81a9](https://qiita.com/mshinoda88/items/9770ee671ea27f2c81a9)<br>
<br>
バウンディングボックスを一意に識別するには4つの変数が必要です。 したがって、イメージ内の物体の各インスタンスについて、次の変数を予測します。

- class_name
- bounding_box_top_left_x_coordinate
- bounding_box_top_left_y_coordinate
- bounding_box_width
- bounding_box_height

つい数年前までは、図14-24 に示すように、1 つの物体を分類し、位置を特定できるよう
に訓練したCNNを画像内でずらして使うのが一般的だった。この例では、画像は6 × 8 のグリッ
ドに分割され、CNN（太い黒線の四角形）が3 × 3 のリージョンをスライドしていく。このCNN
は、画像の左上でもっとも左のバラの一部を検知し、初めて右に1 ステップシフトされたときにも
同じバラを検知する。画像全体でCNNをスライドさせ続けると、すべての3 × 3 のリージョンを
見ることになる。さらに、物体のサイズはさまざまになり得るので、異なるサイズのリージョンも
スライドさせたい。たとえば、3 × 3のリージョンのスライドが終わったら、4 × 4のリージョンに
もCNNをスライドさせる。


![14_24](fig/14_24.png)

このテクニックはわかりやすいが、同じ物体を少し異なる位置で何度も検知することになる。すると、不要なバウンディングボックスを取り除くために後処理が必要になる。よく使われる方法は、次のような仕組みの非極大抑制（NMS：non-max suppression）と呼ばれるものである。

1. まず、画像に本当に花がある確率を推計するために、CNNに新たに物体あり（objectness）の出力を追加する必要がある（「花なし」クラスを追加する方法もあるが、この方法は「物体あり」ほどうまく機能しない）。この出力はシグモイド活性化関数を使わなければならない。また、二項交差エントロピー損失関数で訓練できる。そして「物体あり」のスコアが何らかのしきい値に満たないバウンディングボックスをすべて取り除く。これで実際には花が含まれていないすべてのバウンディングボックスが削除される。
2. 「物体あり」のスコアがもっとも高いバウンディングボックスを見つけ、それと大きく重なり合うほかのすべてのバウンディングボックス（たとえばIoU が60% 以上のもの）を取り除く。たとえば、図14-24では、「物体あり」スコアがもっとも高いバウンディングボックスは、中央
上のバラを囲む太いバウンディングボックスだ（「物体あり」スコアは、バウンディングボックスの太さで表されている）。同じバラのバウンディングボックスはこのバウンディングボックスと大きく重なり合うので、すべて取り除かれる。
3. 取り除くバウンディングボックスがなくなるまで、ステップ2 を繰り返す。

この単純なアプローチによる物体検知は、なかなかよく機能するが、CNNを何度も実行しなけれ
ばならないので、かなり遅い。幸い、画像の上でもっとすばやくCNNをスライドさせる方法があ
る。それは、全層畳み込みネットワーク（FCN：fully convolutional network）を使うものである。

## 14.9.1 全層畳み込みネットワーク（FCN：fully convolutional network）

◆参考<br>
[https://qiita.com/obukoh/items/2916feae33c2f9ca193e](https://qiita.com/obukoh/items/2916feae33c2f9ca193e)

Q1.Fully Convolutional Networkとは何か？<BR>
- Semantic Segmentationにディープラーニングを使った最初の手法がFCN (Fully Convolutional Network)
- Semantic Segmentationは画像をpixel単位でどのクラスに属するか分類する。そのためPixel-labelingとも呼ばれる。
- 一般物体認識を行うニューラルネットワークでは、入力ユニット数が入力画像のサイズ、出力ユニット数は分類するクラス数だった
- Semantic Segmentationでは出力ユニット数は画像サイズ×分類クラス数(入力ユニット数は同様に画像サイズ)
- pixelごとにラベル付けされた教師データを与えて学習することで、入力画像の各pixelがどのクラスに分類されるかの確率を出力できるようになる。
- FCN (Fully Convolutional Network)は、CVPR 2015, PAMI 2016で発表された Fully Convolutional Networks for Semantic Segmentationで提案されたSemantic Segmentation手法。
- `FCNの大きな特長は、全結合層を持たず、ネットワークが畳み込み層のみで構成されていること。`
- つまり、一般物体認識用のCNNをSemantic Segmentation用に改良したということ
- `一般物体認識の畳み込みニューラルネットワークの全結合層を1×1の畳み込み層に置き換えている。CNN最上部の全結合層は畳み込み層に置き換えられる`
- 本来だったら、全結合(Affine層)によって1次元のデータ列になるところを、FCNは画像サイズが1×1(1pixcel)になるように無理やり畳み込んでいる(=これすなわち、フィルタの数分の特徴を示す値が抽出されたということ=(フィルタの数だけ特徴量が取れた))
- 全結合層では、全pixelに対して接続しているノードごとに重みをかける処理を行っていた。これはつまり、ノードの数だけ1×1の畳み込みを行う処理と同等である。(参考: 全結合層（Affine）の仕組を、数式なしでざっくり整理してみた／ディープラーニング)
要するに全結合層でやっていた計算と全く同じことをやっている、ただし、1×1の畳み込み層になるということは全結合層で出力されるのが(行列計算から導き出された)数値であるのに対して、FCNでは畳み込まれた画像(もはや1×1なら数値だが…)である。
特徴マップのサイズはMaxプーリングを経て小さくなっているため、入力画像H×Wに対して特徴マップのサイズはH/32×W/32になっている。
結論 : FCNは一般物体認識用のCNNをSemantic Segmentation用に改良したものである。詳しくは、全結合層が1×1の畳み込み層になっただけである。(やっていることは同じ)




それぞれサイズ
（カーネルサイズではなく、特徴量マップのサイズ）が7 × 7の100個の特徴量マップを出力する畳
み込み層の上部に200 ニューロンの全結合層があるものとする。個々のニューロンは、畳み込み層
の100×7×7のニューロンの活性化の度合い（と、バイアス項）の加重総和を計算する。では、こ
の全結合層の代わりに7 × 7 の200 個のフィルタを持つ"valid"パディングの畳み込み層を使った
らどうなるだろうか。この層はそれぞれ1 × 1 の200 個の特徴量マップを出力する（カーネルが入
力特徴量マップのサイズであり、"valid"パディングを使っているため）。つまり、全結合層と同
じように200 個の数値を出力するわけである。そして、畳み込み層が行う計算をよく見ると、全結
合層が生成する数値とまったく同じだということがわかる。違いは、全結合層の出力が[バッチサ
イズ,200] という形のテンソルであるのに対し、畳み込み層の出力が[バッチサイズ,1,1,200] という
形のテンソルであることだけだ。

このことがなぜ重要なのだろうか。全結合層は特定の入力サイズを要求する（入力特徴量ごとに
1 個の重みを持つ）のに対し、畳み込み層は画像がどのようなサイズでも文句を言わずに処理する
からである（ただし、入力が特定の数のチャネルを持つことは要求する）†27。FCNには畳み込み
層しか含まれていないので（プーリング層もあるが、畳み込み層と特徴が同じである）、どのよう
なサイズの画像でも訓練、予測に使える。
たとえば、花の分類と位置特定のためにすでにCNNを訓練していたとする。訓練には224×224の画像が使われ、10 個の数値が出力される。出力0 から4 はソフトマックス活性化関数に送られ、クラス確率（クラスごとに1つ）が計算される。出力5はロジスティック活性化関数に送られ、「物体あり」スコアの計算に使われる。出力6 から9 は活性化関数を使わず、バウンディングボックスの中央の座標と幅、高さを表す。このCNNの全結合層は畳み込み層に変換できる。
実際、再訓練の必要さえない。全結合層の重みを畳み込み層にコピーするだけでよいのだ。訓練の前にCNNをFCNに変換してもよい。
ここで、出力層の前の最後の畳み込み層（ボトルネック層とも呼ばれる）は、ネットワークに
224 × 224 の画像が与えられると7 × 7 の特徴量マップを出力する（図14-25 の左側）。FCN に
448 × 448の画像を与えると（図14-25 の右側）、ボトルネック層は14 × 14の特徴量マップを出力
する†28。全結合層の代わりに、サイズが7 × 7 の10 個のフィルタと"valid"パディング、ストラ
イド1 の畳み込み層を使っているので、出力はサイズが8 × 8（14 − 7 + 1 = 8）の10 個の特徴量
マップになる。`つまり、FCNは画像全体を1 度で処理し、個々のセルが10 個の数値（5 個がクラス確率、1 個が「物体あり」スコア、4 個がバウンディングボックスの座標とサイズ）を持つ8 × 8のグリッドを出力する。`これは、行あたり8 ステップ、列あたり8 ステップでもとのCNNを画像
全体でスライドさせているのと同じだ。もとの画像を14 × 14 のグリッドに分割し、グリッド全体
で7 × 7 のウィンドウをスライドさせるところを想像してみよう。ウィンドウが取り得る位置は
8 × 8 = 64あるため、8 × 8の予測が得られる。しかし、FCNの場合、ネットワークは同じ画像を
1 度しか見ないので、はるかに効率がよい。実際、次節で取り上げる物体検知アーキテクチャは、
YOLO（You look only once）と呼ばれている。

![14_25](fig/14_25.png)

◆参考<br>
[https://blog.negativemind.com/2019/03/11/semantic-segmentation-by-fully-convolutional-network/](https://blog.negativemind.com/2019/03/11/semantic-segmentation-by-fully-convolutional-network/)
![14_eg1](fig/14_eg1.png)
![14_1](fig/14_1.png)
![14_2](fig/14_2.png)
![14_14](fig/14_14.png)
![14_7](fig/14_7.png)


## 14.9.2 YOLO(You Only Look Once)

◆参考<br>
[https://qiita.com/cv_carnavi/items/68dcda71e90321574a2b](https://qiita.com/cv_carnavi/items/68dcda71e90321574a2b)

[https://qiita.com/cyberailab/items/850806c08af08853bac8](https://qiita.com/cyberailab/items/850806c08af08853bac8)

[https://zenn.dev/opamp/articles/5198d6bf369b8e](https://zenn.dev/opamp/articles/5198d6bf369b8e)

YOLOは、Joseph Redmonらが2015年の論文（https://homl.info/yolo）†29で提案し、2016年
の論文（https://homl.info/yolo2、YOLOv2）†30と2018 年の論文（https://homl.info/yolo3、
YOLOv3）†31で改良したきわめて高速で正確な物体検知アーキテクチャだ。あまりにも高速な
ので、Redmonのデモ（https://homl.info/yolodemo）が示すように、動画をリアルタイムで処理
できる。
YOLOv3のアーキテクチャは、いま、説明したものと近いが、重要な違いがいくつかある。

- グリッドセルごとに5 個のバウンディングボックス（1 個だけではなく）を出力し、それぞ
れに「物体あり」スコアがある。また、グリッドセルごとに20のクラス確率を出力する（ク
ラスが20 種類のPASCAL VOCデータセットを使って訓練されているため）。そのため、グ
リッドセルごとに出力される数値は45個になる（それぞれ4個の座標から構成されるバウン
ディングボックスが5 個で20 個、「物体あり」スコアが5 個、クラス確率が20 個）
- YOLOv3は、バウンディングボックスの中心の絶対座標ではなく、グリッドセルの座標から
のオフセットを予測する。(0, 0) はセルの左上隅、(1, 1) は右下隅を表す。YOLOv3 が個々
のグリッドセルで予測するバウンディングボックスは、中心がそのセルにあるものだけであ
る（ただし、バウンディングボックス自体は、一般にそのグリッドセルの外まで広がってい
る）。YOLOv3は、バウンディングボックスの座標にロジスティック活性化関数を適用し、0
から1 までの範囲に収まるようにしている。
- YOLOv3 は、ニューラルネットを訓練する前に、5 個の代表的なバウンディングボックス
（アンカーボックス：anchor box、またはbounding box prior：優先的なバウンディングボッ
クスと呼ばれる）の位置を見つける。そのために、訓練セットのバウンディングボックスの
高さと幅にK平均法（9 章参照）を適用する。たとえば、訓練セットの画像に多数の歩行者
が含まれていたとすると、アンカーボックスのなかに典型的な歩行者の位置を示すものがあ
るだろう。ニューラルセルがグリッドセルあたり5 個のバウンディングボックスを予測する
ときに実際に予測するのは、個々のアンカーボックスをどの程度スケーリングするかにな
る。たとえば、あるアンカーボックスの高さが100 ピクセル、幅が50 ピクセルで、ネット
ワークの予測は垂直リスケーリングファクタが1.5、水平リスケーリングファクタが0.9 だっ
たとする（グリッドセルのなかのどれか1 つで）。すると、150 × 45 ピクセルのバウンディ
ングボックスを予測したことになる。より正確に言うと、ネットワークは、個々のグリッド
セルと個々のアンカーボックスについて、垂直、水平リスケーリングファクタの対数を予測
する。このようなbounding box prior を用意しておくと、ネットワークが予測するバウン
ディングボックスのサイズは適切なものになりやすく、妥当なバウンディングボックスがど
のようなものになるかを早く学習するので、訓練にかかる時間が短くなる。
- ネットワークが異なるスケーリングの画像で訓練される。ネットワークは、訓練中、数バッ
チを処理すると、無作為に新しい画像サイズを選ぶ（330 × 330 ピクセルから608 × 608 ピ
クセルの間で）。こうすると、ネットワークは異なるスケーリングの物体を検知することを
学習できる。それ以上に大きいのは、YOLOv3 が異なるスケーリングで使えるようになる
ことだ。スケーリングが小さいと、スケーリングが大きいときと比べて正確さは見劣りする
が、高速に処理できるので、ユースケースに合わせて適切なトレードオフを選べる。

YOLOには、CNNで失われる空間解像度（すぐあとでセマンティックセグメンテーションを取
り上げるときに説明する）の一部を回復するスキップ接続の利用など、興味深いイノベーションが
ほかにも含まれている。著者たちは、2016 年の論文で、階層的分類を使うYOLO9000モデルを提案した。このモデルは、WordTree と呼ばれる階層構造の個々のノードに属する確率を予測する。
この方法により、ネットワークは、どの犬種かははっきりしなくても、画像が犬のものであること
を確信を持って予測できるようになる。3 本の論文をすべて読むことをお勧めしたい。とても楽し
く読めるし、深層学習システムを漸進的に改良できることを示す、すばらしい例になっている。

GitHub には、TensorFlow を使って実装された複数のYOLO がある。特にZihao Zang の
TensorFlow 2による実装（https://homl.info/yolotf2）はぜひ見ておきたい。TensorFlow Models
プロジェクトにもオブジェクト検知モデルがあり、その多くには事前学習した重みがついて
いる。さらに、そのなかのSSD（https://homl.info/ssd）†32やFaster R-CNN（https://homl.
info/fasterrcnn）†33は、TF Hubにも移植されている。この2つはかなり広い範囲で使われている。
SSDはYOLOと同様の「単発」の検知モデルだが、Faster R-CNNはもっと複雑である。画像はま
ずCNNに渡され、その出力はさらに物体を含んでいる可能性が高いバウンディングボックスを提
案するRPN（Region Proposal Network）に送られる。そして、CNNが切り取った出力に基づい
て個々のバウンデングボックスに対して分類器を実行する。
どの検知システムを選ぶかは、スピード、正解率、利用できる事前学習済みモデル、訓練時間、
複雑度などのさまざまな要因によって決まる。論文には指標の表が含まれているが、テスト環境に
は多くの変動要因があり、技術進歩のペースは速い。多くの人々の役に立つような、数ヵ月以上に
渡る公平な比較は難しい。
これで物体のまわりにバウンディングボックスを描いて物体を検知できるようになった。しか
し、もっと正確な情報が必要かもしれない。ピクセルレベルでどの物体に属するかを正確に判断す
る方法を見てみよう。

# 14.10 セマンティックセグメンテーション

◆参考<br>
[https://www.skillupai.com/blog/tech/segmentation1/](https://www.skillupai.com/blog/tech/segmentation1/)<br>

セマンティックセグメンテーションは、図14-26 に示すように、属する物体のクラス（たとえば
道路、自動車、歩行者、建物など）に従ってピクセルを分類する。同じクラスの異なる物体は区
別しないことに注意しよう。たとえば、右側のすべての自転車は、分類後の画像では1 つの大きな
ピクセルの塊になってしまう。このタスクの難しいところは、画像を普通のCNNに通してしまう
と、空間解像度が次第に失われていく（2 以上のストライドの層のために）ことだ。つまり、普通
のCNNでは、画像の左下のどこかに人がいることがわかるだけで、それ以上の正確なことは決し
てわからないのである。
物体検知と同様に、この問題を解くためのアプローチは多数あり、なかには複雑なものもある。
しかし、先ほど取り上げたJonathan Long らの2015 年の論文がかなり単純なソリューションを提
案した。著者たちは事前学習済みのCNN からスタートして、それをFCN に変身させた。CNN
は、入力イメージに全部で（つまり、1 よりも大きい分を全部合計して）32 のストライドを適用
している。つまり、最後の層が出力する特徴量マップは、入力イメージの1/32 の大きさしかな
い。これでは明らかに粗すぎる。そこで、著者たちは解像度を32 倍にするアップサンプリング層（upsampling layer）を追加した。

![14_26](fig/14_26.png)

アップサンプリング（画像サイズの拡大）にも、バイリニア補間（bilinear interpolation）をはじ
めとする複数の方法があるが、それでまずまずの結果が得られるのは、せいぜい4倍か8倍までだ。
彼らが使ったのは、転置畳み込み層（transposed convolutional layer）である†34。これは、空行と
空列を挿入して画像を拡大してから通常の畳み込みを実行する（図14-27 参照）。同じことを分数
のストライド（たとえば、図14-27 では1/2）を使った通常の畳み込みだと考える人々もいる。転
置畳み込み層は、線形補間に近いことを実行するように初期化することができるが、訓練可能な層
なので、訓練中にもっとよい方法を学習していく。tf.keras では、Conv2DTranspose層が使える。

転置畳み込み層では、ストライドはフィルタステップのサイズではなく、入力をどの程度引き
伸ばすかを定義する。そのため、ストライドが大きければ大きいほど出力も大きくなる（畳み
込み層やプーリング層のときとは異なる）。

![14_27](fig/14_27.png)

◆参考<br>
[Link](https://www.google.com/search?q=%E3%82%B9%E3%82%AD%E3%83%83%E3%83%97%E6%8E%A5%E7%B6%9A&rlz=1C5CHFA_enJP880JP881&oq=%E3%82%B9%E3%82%AD%E3%83%83%E3%83%97%E6%8E%A5%E7%B6%9A&aqs=chrome..69i57j0i512j0i5i30l5.5499j0j7&sourceid=chrome&ie=UTF-8)

この方法は許容範囲だが、まだ不正確過ぎる。そこで、著者たちは下位層からのスキップ接続を
追加した。たとえば、出力画像を2 倍（32 倍ではなく）にアップサンプリングし、同じように出力
の倍の解像度を持っていた下位層の出力を加算して、それを16 倍にアップサンプリングして32 倍の画像を作った（図14-28 参照）。こうすると、以前のプーリング層によって失われた空間解像度
の一部が復元される。著者たちの最高のアーキテクチャでは、同じようなスキップ接続をもう1 つ
使って、さらに下位の層からの精細なディテールを復元した。つまり、もとのCNNの出力は、×2
のアップスケーリング、下位層の出力の加算（適切なスケールでの）、× 2 のアップスケーリング、
さらに下位の層の出力の加算、× 8 のアップスケーリングという新たな処理を受けるようになっ
た。もとの画像のサイズを越えてさらにアップスケーリングすることもできる。これは画像の解像
度を上げるために使えるテクニックで、超解像技術（super-resolution）と呼ばれている。

![14_28](fig/14_28.png)

GitHub には、TensorFlow によるセマンティックセグメンテーションの実装のリポジトリも多
数含まれている。TensorFlow Models プロジェクトには、事前学習済みのインスタンスセグメン
テーション（instance segmentation）モデルさえある。インスタンスセグメンテーションは、セ
マンティックセグメンテーションと似ているが、同じクラスのすべての物体を1 つの大きな塊にま
とめてしまうのではなく、個々の物体を互いに区別する（たとえば、1 つひとつの自転車を区別す
る）。現在TensorFlow Modelsプロジェクトで公開されているインスタンスセグメンテーションモ
デルは、2017年の論文（https://homl.info/maskrcnn）†35で提案されたMask R-CNN アーキテク
チャに基づいている。Mask R-CNNは、Faster R-CNNを拡張し、個々のバウンディングボックス
のために新たにピクセルマスクを作っている。そのため、個々の物体を囲むバウンディングボック
スと物体の推定クラス確率の集合だけでなく、バウンディングボックス内のオブジェクトに属する
ピクセルの位置を示すピクセルマスクも得られる。
以上見てきたように、深層コンピュータビジョンという分野は広大ですごい勢いで前進してい
る。毎年ありとあらゆる種類のアーキテクチャが生まれており、それらはどれも畳み込みニューラ
ルネットワークを基礎としている。わずか数年間で驚くほどの進歩を遂げており、研究者たちは、
敵対的学習（adversarial learning：ネットワークの裏をかくことを意図して作られた画像に対して
抵抗力のあるネットワークを作る試み）、説明可能性（ネットワークが特定の分類をした理由の理解）、リアルな画像生成（17 章で取り上げる）、シングルショット学習（1 度見ただけで物体を認識
できるシステム）などの難しい問題に研究の軸足を移していっている。Geoffrey Hinton のカプセ
ルネットワーク（capsule network： https://homl.info/capsnet）のようにまったく新しいアーキ
テクチャへの取り組みもある（私はカプセルネットワークを解説する動画を作り、対応するコード
をJupyter ノートブックに書いたことがある。https://homl.info/capsnetvideos を参照）。次章で
は、再帰型ネットワークや畳み込みネットワークを使った時系列データなどのシーケンシャルデー
タの処理方法を見ていく。